In [68]:
# Import libraries necesarry for web scraping with selenium
#import selenium
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service as ChromeService
#import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

import requests
from bs4 import BeautifulSoup
import json
import PyPDF2
import io
import re
import pandas as pd
# Import necessary libraries


Here are the Websites I am thinking of using:

https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi

https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/topic-pages/tables/table-5

https://www.icpsr.umich.edu/web/NACJD/studies/38604/summary

https://bjs.ojp.gov/library/publications/criminal-victimization-2022

https://endrapeoncampus.org/map-and-tool/

https://ope.ed.gov/campussafety/#/compare/confirm

https://www.prri.org/research/2020-census-of-american-religion/

https://www.statista.com/statistics/221454/share-of-religious-americans-by-state/

BIG 12 (for 2024 group) FIRE & SAFETY REPORTS:

BYU https://police.byu.edu/2023-asr-provo  (page 9)

Baylor https://www.baylor.edu/risk/doc.php/399779.pdf (page 12)

Cinncinati https://www.uc.edu/content/dam/refresh/publicsafety-62/docs/ucpd/2023-annual-security-report.pdf (page 34)

Houston http://m2s-conf.uh.edu/police/records-reports/annualreport/2023-uh-annual-security-and-fire-safety-report.pdf (page 39)

Iowa https://www.police.iastate.edu/wp-content/uploads/2023/09/2022-Report-Final.pdf (page 6)

KU https://civilrights.ku.edu/sites/civilrights/files/documents/Reports/University%20of%20Kansas%20Annual%20Security%20and%20Fire%20Safety%20Report-2023-KLETC.pdf (page 60)

K-state https://www.k-state.edu/report/clery/reports/KSUCleryReport2023.pdf (page 13)

OSU https://safety.okstate.edu/police/documents/annual-security-reports/2023-annual-safety-report.pdf (page 37)

OU https://ou.edu/content/dam/OUPD/documents/safety.pdf (page 52)

TCU https://police.tcu.edu/wp-content/uploads/2023/09/TCU-Annual-Security-Report-Fire-Safety-Report_2023.v1.pdf (page 80)

Texas Tech https://www.depts.ttu.edu/clery/reports/2023LubbockASRfinal.pdf (page 107)

UCF https://police.ucf.edu/sites/default/files/2023%20Annual%20Security%20%26%20Fire%20Safety%20Report.pdf (page 78)

UTA https://compliance.utexas.edu/sites/default/files/documents/2023-annual-security-fire-safety-report_0.pdf (page 80)

West Virginia https://police.wvu.edu/files/d/a1a0a6fe-444e-4353-8e92-7dcbeb00cd27/2023-annual-security-and-fire-safety-report-with-appendix.pdf (page 36)

In [2]:

import camelot
import pandas as pd

In [4]:
BYU_table = camelot.read_pdf('BYU_report.pdf', pages='9,10', flavor='stream')
Baylor_tables = camelot.read_pdf('Baylor_report.pdf', pages='12,13', flavor='stream')
Baylor_table_VAWA = camelot.read_pdf('Baylor_report.pdf', pages='13', flavor='stream', table_regions=['0,400,800,0'])
Cinn_table = camelot.read_pdf('Cinn_report.pdf', pages='34', flavor='stream')
Iowa_table = camelot.read_pdf('Iowa_report.pdf', pages='6', flavor='stream')
Kstate_table = camelot.read_pdf('Kstate_report.pdf', pages='13', flavor='stream', table_regions=['0,600,800,0'])
KU_tables = camelot.read_pdf('KU_report.pdf', pages='60,61', flavor='stream')
OSU_tables = camelot.read_pdf('OSU_report.pdf', pages='37,38', flavor='stream')
OU_tables = camelot.read_pdf('OU_report.pdf', pages='52', flavor='stream')
OU_table_VAWA = camelot.read_pdf('OU_report.pdf', pages='53', flavor='stream', table_regions=['0,400,500,350'])
TCU_tables = camelot.read_pdf('TCU_report.pdf', pages='80,81', flavor='stream')
Ttech_tables =camelot.read_pdf('Ttech_report.pdf', pages='107,109,111', flavor='stream',table_regions=['0,200,800,0'])
UCF_tables = camelot.read_pdf('UCF_report.pdf', pages='78,79,80', flavor='stream')
UCF_tables_VAWA = camelot.read_pdf('UCF_report.pdf', pages='78,79,80', flavor='stream', table_regions=['0,600,800,0'])
UH_table = camelot.read_pdf('UH_report.pdf', pages='39', flavor='stream')
UH_table_VAWA = camelot.read_pdf('UH_report.pdf', pages='41', flavor='stream')
UTA_tables = camelot.read_pdf('UTA_report.pdf', pages='80, 81', flavor='stream')
WV_table = camelot.read_pdf('WV_report.pdf', pages='36', flavor='stream')
WV_table_VAWA = camelot.read_pdf('WV_report.pdf', pages='39', flavor='stream')

In [6]:
### BYU ###
# take only rows we need
BYU_sexOff = BYU_table[0].df.iloc[[1,9,10,11,12]]
BYU_VAWA = BYU_table[1].df.iloc[[2,4,5,6]]
# take out columns we don't need
BYU_sexOff = BYU_sexOff.drop([2,4,6], axis=1)
BYU_VAWA = BYU_VAWA.drop([2,4,6], axis=1)
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
BYU_VAWA = BYU_VAWA.reset_index(drop=True)
# make the first row the column names
BYU_sexOff.columns = BYU_sexOff.iloc[0]
BYU_VAWA.columns = BYU_VAWA.iloc[0]
# drop the first row
BYU_sexOff = BYU_sexOff.drop([0])
BYU_VAWA = BYU_VAWA.drop([0])
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
BYU_VAWA = BYU_VAWA.reset_index(drop=True)
# Adding the year counts together
BYU_sexOff['2020 Total'] = BYU_sexOff['2020'].astype(int).sum(axis = 1)
BYU_sexOff['2021 Total'] = BYU_sexOff['2021'].astype(int).sum(axis = 1)
BYU_sexOff['2022 Total'] = BYU_sexOff['2022'].astype(int).sum(axis = 1)
BYU_VAWA['2020 Total'] = BYU_VAWA['2020'].astype(int).sum(axis = 1)
BYU_VAWA['2021 Total'] = BYU_VAWA['2021'].astype(int).sum(axis = 1)
BYU_VAWA['2022 Total'] = BYU_VAWA['2022'].astype(int).sum(axis = 1)
# Add the school name
BYU_sexOff['School'] = 'BYU'
BYU_VAWA['School'] = 'BYU'
# Keep only the columns we want
BYU_sexOff = BYU_sexOff[['Year Reported','School','2020 Total','2021 Total','2022 Total']]
BYU_VAWA = BYU_VAWA[['Year Reported','School','2020 Total','2021 Total','2022 Total']]
# Rename Year Reported to Offense
BYU_sexOff = BYU_sexOff.rename(columns={'Year Reported': 'Offense'})
BYU_VAWA = BYU_VAWA.rename(columns={'Year Reported': 'Offense'})

# Combine the two BYU tables
BYU_sexOff = pd.concat([BYU_sexOff,BYU_VAWA])
# Reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
#BYU_sexOff

In [8]:
### Baylor University ###
# Only take row 16 and below
Baylor_sexOff = Baylor_tables[0].df[16:]
Baylor_sexOff = Baylor_sexOff.drop([19], axis=0)
# reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
# Remove the 0 and 2 columns
Baylor_sexOff = Baylor_sexOff.drop([0,2], axis=1)
# Add a row with custom values
data = {1:'Statutory Rape', 3:'0',4:'0',5:'0',6:'0',7:'0',8:'0'}
# Append the row to the dataframe
Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)
# only keep columns 1,6
Baylor_sexOff = Baylor_sexOff[[1,6]] 
# Rename the columns
Baylor_sexOff = Baylor_sexOff.rename(columns={1: 'Offense', 6: '2020 Total'}) 
# remove the first row
Baylor_sexOff = Baylor_sexOff.drop([0], axis=0) 

Baylor_sexOff['School'] = 'BU'
Baylor_sexOff['2021 Total'] = ['20','10','0','0']
Baylor_sexOff['2022 Total'] = ['22','8','0','0']

# Reorder the columns
Baylor_sexOff = Baylor_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
Baylor_sexOff

# Set up Baylor VAWA
Baylor_VAWA = Baylor_table_VAWA[0].df[[1,2,6]]
# Remove rows 0 5 and 7
Baylor_VAWA = Baylor_VAWA.drop([0,5,7], axis=0)
# Fill the First column with the offense
Baylor_VAWA[1] = ['Domestic Violence','Domestic Violence','Domestic Violence','Dating Violence','Dating Violence','Dating Violence','Stalking','Stalking','Stalking']
# Rename the columns
Baylor_VAWA = Baylor_VAWA.rename(columns={1: 'Offense', 2: 'Year', 6: 'Total'})
# Pivot the table, keeping Offense in as a column
Baylor_VAWA = Baylor_VAWA.pivot(index='Offense', columns='Year', values='Total')
# Reset the index
Baylor_VAWA = Baylor_VAWA.reset_index()
# Add the school name
Baylor_VAWA['School'] = 'BU'
# Reorder the columns
Baylor_VAWA = Baylor_VAWA[['Offense','School','2020','2021','2022']]
# Rename the columns
Baylor_VAWA = Baylor_VAWA.rename(columns={'2020': '2020 Total','2021': '2021 Total','2022': '2022 Total'})
# Rename the index
Baylor_VAWA = Baylor_VAWA.rename_axis(None, axis=1)

# Combine the two Baylor tables
Baylor_sexOff = pd.concat([Baylor_sexOff,Baylor_VAWA])
# switch rows 4 and 5
Baylor_sexOff.iloc[[4,5]] = Baylor_sexOff.iloc[[5,4]] 
# Reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)

#Baylor_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\2640987900.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)


In [10]:
### Cinncinati University ###
# Keep rows 3, 6-9: Sex offenses, 15-17: Stalking/dating violence
Cinn_sexOff = Cinn_table[0].df.iloc[[3,6,7,8,9,15,16,17]]
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Make first row the columns
Cinn_sexOff.columns = Cinn_sexOff.iloc[0]
# Drop the first row
Cinn_sexOff = Cinn_sexOff.drop([0])
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Remove all the * from row 1
Cinn_sexOff.iloc[0,] = Cinn_sexOff.iloc[0,].str.replace('*','',regex=True)
# Add the school name
Cinn_sexOff['School'] = 'CU'
# Sum the columns
Cinn_sexOff['2020 Total'] = Cinn_sexOff['2020'].astype(int).sum(axis = 1)
Cinn_sexOff['2021 Total'] = Cinn_sexOff['2021'].astype(int).sum(axis = 1)
Cinn_sexOff['2022 Total'] = Cinn_sexOff['2022'].astype(int).sum(axis = 1)
# Keep only the columns we want
Cinn_sexOff = Cinn_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Swap rows 3 and 4
Cinn_sexOff.iloc[[2,3]] = Cinn_sexOff.iloc[[3,2]]
#Cinn_sexOff

In [12]:
### Iowa State ###
# Keep idex rows 0,1, 8-11
Iowa_sexOff = Iowa_table[0].df.iloc[[0,1,8,9,10,11,36,38,39]]
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Keep columns 0,16,17,18
Iowa_sexOff = Iowa_sexOff[[0,16,17,18]]
# Drop row 0 and 1
Iowa_sexOff = Iowa_sexOff.drop([0,1], axis=0)
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Rename the columns
Iowa_sexOff = Iowa_sexOff.rename(columns={0: 'Offense', 16: '2020 Total', 17: '2021 Total', 18: '2022 Total'})
# Add the school name
Iowa_sexOff['School'] = 'ISU'
# Reorder the columns
Iowa_sexOff = Iowa_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Rename Offense in row 4 and 5
Iowa_sexOff.iloc[4,0] = 'Dating Violence'
Iowa_sexOff.iloc[5,0] = 'Domestic Violence'
# Swap rows 4 and 5
Iowa_sexOff.iloc[[4,5]] = Iowa_sexOff.iloc[[5,4]]

#Iowa_sexOff

In [14]:
### Kansas State ###
# Keep idex rows 9, 17-20: Sex offenses, 22-24: Stalking/dating violence
Kstate_sexOff = Kstate_table[0].df.iloc[[17,18,19,20,22,23,24]]
# Rename the columns
Kstate_sexOff.columns = ['Offense','2020','2020','2020','2020','2021','2021','2021','2021','2022','2022','2022','2022']
# Make yearly totals
Kstate_sexOff['2020 Total'] = Kstate_sexOff['2020'].astype(int).sum(axis = 1)
Kstate_sexOff['2021 Total'] = Kstate_sexOff['2021'].astype(int).sum(axis = 1)
Kstate_sexOff['2022 Total'] = Kstate_sexOff['2022'].astype(int).sum(axis = 1)
# Make school column
Kstate_sexOff['School'] = 'KSU'
# Keep only the columns we want
Kstate_sexOff = Kstate_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Drop the first row
#Kstate_sexOff = Kstate_sexOff.drop([0], axis=0)
# Reset the index
Kstate_sexOff = Kstate_sexOff.reset_index(drop=True)

#Kstate_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\463352260.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Kstate_sexOff['2020 Total'] = Kstate_sexOff['2020'].astype(int).sum(axis = 1)
C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\463352260.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Kstate_sexOff['2021 Total'] = Kstate_sexOff['2021'].astype(int).sum(axis = 1)
C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\463352260.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [16]:
### Kansas University ###
# Create a data frame from scratch that matches the BYU_sexOff data frame for the KU data
KU_scratch = pd.DataFrame(columns=['Offense','School','2020 Total','2021 Total','2022 Total'])
# Add this row to the data frame
KU_data = {'Offense':'Rape', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Fondling', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'1'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Incest', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Statutory Rape', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Domestic Violence', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Dating Violence', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Stalking', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'1'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)

KU_sexOff = KU_scratch
#KU_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\3469329722.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\3469329722.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\3469329722.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\3469329722.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = 

In [18]:
### Oklahoma State University ###
# Keep idex rows 0 9-20
OSU_sexOff = OSU_tables[0].df.iloc[[9,10,11,12,13,14,15,16,17,18,19,20]]
OSU_VAWA = OSU_tables[1].df.iloc[[7,8,9,10,11,12,13,14,15]]
# Keep index columns 0,1,5
OSU_sexOff = OSU_sexOff[[0,1,5]]
OSU_VAWA = OSU_VAWA[[0,1,5]]
# Fill in the rest of column 0
OSU_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
OSU_VAWA['Offense'] = ['Dating Violence', 'Dating Violence', 'Dating Violence', 'Domestic Violence', 'Domestic Violence', 'Domestic Violence', 'Stalking', 'Stalking', 'Stalking']
# Replace Column 0 with the Offense column
OSU_sexOff = OSU_sexOff[['Offense',1,5]]
OSU_VAWA = OSU_VAWA[['Offense',1,5]]
# Pivot the tables
OSU_sexOff = OSU_sexOff.pivot(columns=1, index='Offense')
OSU_sexOff.columns = OSU_sexOff.columns.droplevel()
OSU_sexOff = OSU_sexOff.reset_index()
OSU_sexOff = OSU_sexOff.rename_axis(None, axis=1)

OSU_VAWA = OSU_VAWA.pivot(columns=1, index='Offense')
OSU_VAWA.columns = OSU_VAWA.columns.droplevel()
OSU_VAWA = OSU_VAWA.reset_index()
OSU_VAWA = OSU_VAWA.rename_axis(None, axis=1)

# Add the school name
OSU_sexOff['School'] = 'OSU'
OSU_VAWA['School'] = 'OSU'
# Rename the columns and Reorder the columns
OSU_sexOff = OSU_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
OSU_VAWA = OSU_VAWA.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
OSU_sexOff = OSU_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
OSU_VAWA = OSU_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
OSU_sexOff = OSU_sexOff.iloc[[2,0,1,3]]
OSU_VAWA = OSU_VAWA.iloc[[1,0,2]]
# Reset the index
OSU_sexOff = OSU_sexOff.reset_index(drop=True)
OSU_VAWA = OSU_VAWA.reset_index(drop=True)
# Combine the two OSU tables
OSU_sexOff = pd.concat([OSU_sexOff,OSU_VAWA])

#OSU_sexOff

In [81]:
### University of Oklahoma ###
# Keep idex rows 4 9-12
OU_sexOff = OU_tables[0].df.iloc[[4,9,10,11,12]]
# Keep Columns 0,1,2,3,7-12
OU_sexOff = OU_sexOff[[0,1,2,3,7,8,9,10,11,12]]
# Make the first row the column names
OU_sexOff.columns = OU_sexOff.iloc[0]
# Drop row 4
OU_sexOff = OU_sexOff.drop([4])
# Combine all the year columns into totals
OU_sexOff['2020 Total'] = OU_sexOff['2020'].astype(int).sum(axis = 1)
OU_sexOff['2021 Total'] = OU_sexOff['2021'].astype(int).sum(axis = 1)
OU_sexOff['2022 Total'] = OU_sexOff['2022'].astype(int).sum(axis = 1)
# Add the school name
OU_sexOff['School'] = 'OU'
# Only use the totals and school columns
OU_sexOff = OU_sexOff[['','School','2020 Total','2021 Total','2022 Total']]
# Rename the columns
OU_sexOff = OU_sexOff.rename(columns={'': 'Offense'})
## VAWA ##
OU_VAWA = OU_table_VAWA[0].df.iloc[[20,21,22,23,24]]
# Remove row 21
OU_VAWA = OU_VAWA.drop([21], axis=0)
# Split column 1 based on the \n
OU_VAWA_1 = OU_VAWA[0].str.split('\n', expand=True)
# Drop Column 4
OU_VAWA_1 = OU_VAWA_1.drop([4], axis=1)
# Rename the columns
OU_VAWA_1.columns = ['Offense','2020','2021','2022']
# Split column 2 based on the \n
OU_VAWA_2 = OU_VAWA[1].str.split('\n', expand=True)
# Drop the first 2 columns
OU_VAWA_2 = OU_VAWA_2.drop([0,1], axis=1)
# Make the first row the column names
OU_VAWA_2.columns = OU_VAWA_2.iloc[0]
# Combine the two columns
OU_VAWA = pd.concat([OU_VAWA_1,OU_VAWA_2], axis=1)
# Reset the index
OU_VAWA = OU_VAWA.reset_index(drop=True)
# drop the first row
OU_VAWA = OU_VAWA.drop([0], axis=0)
# Add the school name
OU_VAWA['School'] = 'OU'
# Calculate the totals
OU_VAWA['2020 Total'] = OU_VAWA['2020'].astype(int).sum(axis = 1)
OU_VAWA['2021 Total'] = OU_VAWA['2021'].astype(int).sum(axis = 1)
OU_VAWA['2022 Total'] = OU_VAWA['2022'].astype(int).sum(axis = 1)
# Keep only the columns we want
OU_VAWA = OU_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]

# Combine the two OU tables
OU_sexOff = pd.concat([OU_sexOff,OU_VAWA])
OU_sexOff = OU_sexOff.reset_index(drop=True)
#OU_VAWA
#OU_sexOff

In [82]:
### Texas Christian University ###
# Keep rows 11-23: Sex offenses
TCU_sexOff = TCU_tables[0].df.iloc[[11,12,13,14,15,16,17,18,19,20,21,22]]
TCU_VAWA = TCU_tables[1].df.iloc[[5,6,7,8,9,10,11,12,13]]
# Dop columns 3 and 5
TCU_sexOff = TCU_sexOff.drop([3,5], axis=1)
TCU_VAWA = TCU_VAWA.drop([3,5], axis=1)
# Fill in the rest of column 0
TCU_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
TCU_VAWA['Offense'] = ['Dating Violence', 'Dating Violence', 'Dating Violence', 'Domestic Violence', 'Domestic Violence', 'Domestic Violence', 'Stalking', 'Stalking', 'Stalking']
# Replace Column 0 with the Offense column
TCU_sexOff = TCU_sexOff[['Offense',1,2,4]]
TCU_VAWA = TCU_VAWA[['Offense',1,2,4]]
# Pivot the tables
TCU_sexOff = TCU_sexOff.pivot(columns=1, index='Offense')
TCU_sexOff.columns = TCU_sexOff.columns.droplevel()
TCU_sexOff = TCU_sexOff.reset_index()
TCU_sexOff = TCU_sexOff.rename_axis(None, axis=1)
TCU_VAWA = TCU_VAWA.pivot(columns=1, index='Offense')
TCU_VAWA.columns = TCU_VAWA.columns.droplevel()
TCU_VAWA = TCU_VAWA.reset_index()
TCU_VAWA = TCU_VAWA.rename_axis(None, axis=1)
# Remove the ^ from the columns
TCU_sexOff.iloc[0,2] = '8'
TCU_VAWA.iloc[1,2] = '15'
# Make yearly totals
TCU_sexOff['2020 Total'] = TCU_sexOff['2020'].astype(int).sum(axis = 1)
TCU_sexOff['2021 Total'] = TCU_sexOff['2021'].astype(int).sum(axis = 1)
TCU_sexOff['2022 Total'] = TCU_sexOff['2022'].astype(int).sum(axis = 1)
TCU_VAWA['2020 Total'] = TCU_VAWA['2020'].astype(int).sum(axis = 1)
TCU_VAWA['2021 Total'] = TCU_VAWA['2021'].astype(int).sum(axis = 1)
TCU_VAWA['2022 Total'] = TCU_VAWA['2022'].astype(int).sum(axis = 1)
# Add the school name
TCU_sexOff['School'] = 'TCU'
TCU_VAWA['School'] = 'TCU'
# Keep only the columns we want
TCU_sexOff = TCU_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
TCU_VAWA = TCU_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
TCU_sexOff = TCU_sexOff.iloc[[2,0,1,3]]
TCU_VAWA = TCU_VAWA.iloc[[1,0,2]]
# Reset the index
TCU_sexOff = TCU_sexOff.reset_index(drop=True)
TCU_VAWA = TCU_VAWA.reset_index(drop=True)
# Combine the Tables
TCU_sexOff = pd.concat([TCU_sexOff,TCU_VAWA])

#TCU_sexOff

In [85]:
### Texas Tech University ###
# Keep rows 10-13
Ttech_sexOff_22 = Ttech_tables[0].df.iloc[[12,13,14,16,36,37,38]]
# Keep columns 0 and 5
Ttech_sexOff_22 = Ttech_sexOff_22[[0,5]]

# Keep rows 10-13
Ttech_sexOff_21 = Ttech_tables[1].df.iloc[[12,13,14,16,34,35,36]]
# Keep columns 0 and 5
Ttech_sexOff_21 = Ttech_sexOff_21[[0,5]]

# Keep rows 10-13
Ttech_sexOff_20 = Ttech_tables[2].df.iloc[[12,13,14,16,34,35,36]]
# Keep columns 0 and 5
Ttech_sexOff_20 = Ttech_sexOff_20[[0,5]]

Ttech_sexOff_20.rename(columns={0:'Offense', 5:'2020 Total'}, inplace=True) 
Ttech_sexOff_22.rename(columns={0:'Offense', 5:'2022 Total'}, inplace=True)
Ttech_sexOff_21.rename(columns={0:'Offense', 5:'2021 Total'}, inplace=True)
# Join the three tables together on the Offense column
Ttech_sexOff = Ttech_sexOff_20.merge(Ttech_sexOff_21, on='Offense', how='outer').merge(Ttech_sexOff_22, on='Offense', how='outer')

# Add the school name
Ttech_sexOff['School'] = 'TTU'
# Rename the rows
Ttech_sexOff['Offense'] = ['Rape','Fondling','Incest','Statutory Rape', 'Dating Violence', 'Domestic Violence', 'Stalking']
# Reorder the columns
Ttech_sexOff = Ttech_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]

# Swap rows 4 and 5
Ttech_sexOff.iloc[[4,5]] = Ttech_sexOff.iloc[[5,4]]

#Ttech_sexOff

In [86]:
### University of Central Florida ###
# Keep rows 8-11
UCF_sexOff_22 = UCF_tables[0].df.iloc[[8,9,10,11]]
UCF_sexOff_21 = UCF_tables[1].df.iloc[[8,9,10,11]]
UCF_sexOff_20 = UCF_tables[2].df.iloc[[8,9,10,11]]
# VAWA
UCF_VAWA_22 = UCF_tables_VAWA[0].df.iloc[[7,8,9]]
UCF_VAWA_21 = UCF_tables_VAWA[1].df.iloc[[7,8,9]]
UCF_VAWA_20 = UCF_tables_VAWA[2].df.iloc[[7,8,9]]
# Drop columns 2 and 4
UCF_sexOff_20 = UCF_sexOff_20.drop([2,4], axis=1)
UCF_sexOff_22 = UCF_sexOff_22.drop([2,4], axis=1)
UCF_sexOff_21 = UCF_sexOff_21.drop([2,4], axis=1)
# VAWA
UCF_VAWA_20 = UCF_VAWA_20.drop([2,4], axis=1)
UCF_VAWA_22 = UCF_VAWA_22.drop([2,4], axis=1)
UCF_VAWA_21 = UCF_VAWA_21.drop([2,4], axis=1)
# Rename columns
UCF_sexOff_22.columns = ['Offense','2022', '2022']
UCF_sexOff_21.columns = ['Offense','2021', '2021']
UCF_sexOff_20.columns = ['Offense','2020', '2020']
# VAWA
UCF_VAWA_22.columns = ['Offense','2022', '2022']
UCF_VAWA_21.columns = ['Offense','2021', '2021']
UCF_VAWA_20.columns = ['Offense','2020', '2020']
# Make yearly totals
UCF_sexOff_20['2020 Total'] = UCF_sexOff_20['2020'].astype(int).sum(axis = 1)
UCF_sexOff_21['2021 Total'] = UCF_sexOff_21['2021'].astype(int).sum(axis = 1)
UCF_sexOff_22['2022 Total'] = UCF_sexOff_22['2022'].astype(int).sum(axis = 1)
# VAWA
UCF_VAWA_20['2020 Total'] = UCF_VAWA_20['2020'].astype(int).sum(axis = 1)
UCF_VAWA_21['2021 Total'] = UCF_VAWA_21['2021'].astype(int).sum(axis = 1)
UCF_VAWA_22['2022 Total'] = UCF_VAWA_22['2022'].astype(int).sum(axis = 1)
# Keep the columns we want
UCF_sexOff_20 = UCF_sexOff_20[['Offense','2020 Total']]
UCF_sexOff_21 = UCF_sexOff_21[['Offense','2021 Total']]
UCF_sexOff_22 = UCF_sexOff_22[['Offense','2022 Total']]
# VAWA
UCF_VAWA_20 = UCF_VAWA_20[['Offense','2020 Total']]
UCF_VAWA_21 = UCF_VAWA_21[['Offense','2021 Total']]
UCF_VAWA_22 = UCF_VAWA_22[['Offense','2022 Total']]
# Join the tables together
UCF_sexOff = UCF_sexOff_20.merge(UCF_sexOff_21, on='Offense', how='outer').merge(UCF_sexOff_22, on='Offense', how='outer')
# VAWA
UCF_VAWA = UCF_VAWA_20.merge(UCF_VAWA_21, on='Offense', how='outer').merge(UCF_VAWA_22, on='Offense', how='outer')
# Add the school name
UCF_sexOff['School'] = 'UCF'
UCF_VAWA['School'] = 'UCF'
# Reorder the columns
UCF_sexOff = UCF_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
UCF_VAWA = UCF_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Combine the two tables
UCF_sexOff = pd.concat([UCF_sexOff,UCF_VAWA])

#UCF_sexOff

In [87]:
### University of Houston ###
# Keep rows  13-26
UH_sexOff = UH_table[0].df.iloc[[14,15,16,17,18,19,20,21,22,23,24,25]]
# VAWA
UH_VAWA = UH_table_VAWA[0].df.iloc[[5,6,7,8,9,10,11,12,13]]
# Keep columns 0,1,6
UH_sexOff = UH_sexOff[[0,1,6]]
UH_VAWA = UH_VAWA[[0,1,6]]
# Fill in the rest of column 0
UH_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
UH_VAWA['Offense'] = ['Dating Violence', 'Dating Violence', 'Dating Violence', 'Domestic Violence', 'Domestic Violence', 'Domestic Violence', 'Stalking', 'Stalking', 'Stalking']
# Replace Column 0 with the Offense column
UH_sexOff = UH_sexOff[['Offense',1,6]]
UH_VAWA = UH_VAWA[['Offense',1,6]]
UH_VAWA.iloc[4,1] = '2021'
# Pivot the table
UH_sexOff = UH_sexOff.pivot(columns=1, index='Offense')
UH_sexOff.columns = UH_sexOff.columns.droplevel()
UH_sexOff = UH_sexOff.reset_index()
UH_sexOff = UH_sexOff.rename_axis(None, axis=1)
# VAWA
UH_VAWA = UH_VAWA.pivot(columns=1, index='Offense')
UH_VAWA.columns = UH_VAWA.columns.droplevel()
UH_VAWA = UH_VAWA.reset_index()
UH_VAWA = UH_VAWA.rename_axis(None, axis=1)
# Add the school name
UH_sexOff['School'] = 'UH'
UH_VAWA['School'] = 'UH'
# Rename the columns and Reorder the columns
UH_sexOff = UH_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
UH_sexOff = UH_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
UH_VAWA = UH_VAWA.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
UH_VAWA = UH_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
UH_sexOff = UH_sexOff.iloc[[2,0,1,3]]
UH_VAWA = UH_VAWA.iloc[[1,0,2]]
# Combine the tables
UH_sexOff = pd.concat([UH_sexOff,UH_VAWA])
# Reset the index
UH_sexOff = UH_sexOff.reset_index(drop=True)

#UH_sexOff

In [107]:
### University of Texas Austin ###
UTA_sexoff = UTA_tables[0].df.iloc[[16,17,18,19,20,21,22,23,24,25,26,27]]
UTA_VAWA = UTA_tables[1].df.iloc[[8,9,10,11,12,13,14,15,16]]
# Drop column 3
UTA_sexoff = UTA_sexoff.drop([3], axis=1)
UTA_VAWA = UTA_VAWA.drop([3], axis=1)
# Fill in the rest of the offense column
UTA_sexoff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
UTA_VAWA['Offense'] = ['Dating Violence', 'Dating Violence', 'Dating Violence', 'Domestic Violence', 'Domestic Violence', 'Domestic Violence', 'Stalking', 'Stalking', 'Stalking']
# Select the columns we want
UTA_sexoff = UTA_sexoff[['Offense',1,2,4,5]]
UTA_VAWA = UTA_VAWA[['Offense',1,2,4,5]]
# Pivot the tables
UTA_sexoff = UTA_sexoff.pivot(columns=1, index='Offense')
UTA_sexoff.columns = UTA_sexoff.columns.droplevel()
UTA_sexoff = UTA_sexoff.reset_index()
UTA_sexoff = UTA_sexoff.rename_axis(None, axis=1)

UTA_VAWA = UTA_VAWA.pivot(columns=1, index='Offense')
UTA_VAWA.columns = UTA_VAWA.columns.droplevel()
UTA_VAWA = UTA_VAWA.reset_index()
UTA_VAWA = UTA_VAWA.rename_axis(None, axis=1)
# Make yearly totals
UTA_sexoff['2020 Total'] = UTA_sexoff['2020'].astype(int).sum(axis = 1)
UTA_sexoff['2021 Total'] = UTA_sexoff['2021'].astype(int).sum(axis = 1)
UTA_sexoff['2022 Total'] = UTA_sexoff['2022'].astype(int).sum(axis = 1)
UTA_VAWA['2020 Total'] = UTA_VAWA['2020'].astype(int).sum(axis = 1)
UTA_VAWA['2021 Total'] = UTA_VAWA['2021'].astype(int).sum(axis = 1)
UTA_VAWA['2022 Total'] = UTA_VAWA['2022'].astype(int).sum(axis = 1)
# Add the school name
UTA_sexoff['School'] = 'UTA'
UTA_VAWA['School'] = 'UTA'
# Keep only the columns we want
UTA_sexoff = UTA_sexoff[['Offense','School','2020 Total','2021 Total','2022 Total']]
UTA_VAWA = UTA_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]

# Combine the two tables
UTA_sexoff = pd.concat([UTA_sexoff,UTA_VAWA])
UTA_sexoff = UTA_sexoff.reset_index(drop=True)

# Reorder the rows
UTA_sexoff = UTA_sexoff.iloc[[2,0,1,3,5,4,6]]
UTA_sexoff = UTA_sexoff.reset_index(drop=True)
#UTA_sexoff


In [88]:
### West Virginia University ###
# Keep rows 15-21
WV_sexOff = WV_table[0].df.iloc[[15,16,17,18,19,20]]
WV_sexOff.iloc[0,3] = '19'
WV_sexOff.iloc[3,2] = '7'
# Keep only columns 0,1,6
WV_sexOff = WV_sexOff[[0,1,6]]
# VAWA
WV_VAWA = WV_table_VAWA[0].df.iloc[[6,7,8,9,10,11,12,13,14]]
WV_VAWA.iloc[6,2] = '22'
# Keep only columns 0,1,6
WV_VAWA = WV_VAWA[[0,1,6]]
# Fill in the rest of column 0
WV_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling']
WV_VAWA['Offense'] = ['Domestic Violence', 'Domestic Violence', 'Domestic Violence', 'Dating Violence', 'Dating Violence', 'Dating Violence', 'Stalking', 'Stalking', 'Stalking']
# Replace Column 0 with the Offense column
WV_sexOff = WV_sexOff[['Offense',1,6]]
WV_VAWA = WV_VAWA[['Offense',1,6]]
# Pivot the table
WV_sexOff = WV_sexOff.pivot(columns=1, index='Offense')
WV_sexOff.columns = WV_sexOff.columns.droplevel()
WV_sexOff = WV_sexOff.reset_index()
WV_sexOff = WV_sexOff.rename_axis(None, axis=1)
#VAWA
WV_VAWA = WV_VAWA.pivot(columns=1, index='Offense')
WV_VAWA.columns = WV_VAWA.columns.droplevel()
WV_VAWA = WV_VAWA.reset_index()
WV_VAWA = WV_VAWA.rename_axis(None, axis=1)
# Add the school name
WV_sexOff['School'] = 'WV'
WV_VAWA['School'] = 'WV'
# Rename the columns and Reorder the columns
WV_sexOff = WV_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
WV_sexOff = WV_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# VAWA
WV_VAWA = WV_VAWA.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
WV_VAWA = WV_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Fill in the empty Data
WV_data = {'Offense':'Incest', 'School': 'WV', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
WV_data = {'Offense':'Statutory Rape', 'School': 'WV', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
# Reorder the rows
WV_sexOff = WV_sexOff.iloc[[1,0,2,3]]
WV_VAWA = WV_VAWA.iloc[[1,0,2]]
# Combine the tables
WV_sexOff = pd.concat([WV_sexOff,WV_VAWA])
# Reset the index
WV_sexOff = WV_sexOff.reset_index(drop=True)

#WV_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\3919744635.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WV_sexOff.iloc[0,3] = '19'
C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\3919744635.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WV_sexOff.iloc[3,2] = '7'
C:\Users\hermi\AppData\Local\Temp\ipykernel_24808\3919744635.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WV_VAWA.iloc[6,2] = '22'
C:\Users\hermi\AppDat

In [109]:

# Creat the Master Table by Joining all the sexOff tables together

# combine the dataframes
master_table = pd.concat([BYU_sexOff,
                          Baylor_sexOff,
                          Cinn_sexOff,
                          Iowa_sexOff,
                          Kstate_sexOff,
                          KU_sexOff,
                          OSU_sexOff,
                          OU_sexOff,
                          TCU_sexOff,
                          Ttech_sexOff,
                          UCF_sexOff,
                          UH_sexOff,
                          UTA_sexoff,
                          WV_sexOff], ignore_index=True)

# Write out the master table to a csv
# master_table.to_csv('Big12_Sex_Dating_Offenses.csv', index=False)
